# Journal


## 1. Decide What To Do First When Approaching My First Full-Stacked Software Development
I am not a professional software developer. My background is primarily in data science, and before this project, the only languages I was comfortable with were Python, SQL, and R. I had little to no experience with frontend development technologies such as HTML, JavaScript, and Typescript, nor did I fully understand how API worked or how frontend and backend communicate with each other.

Despite these gaps, the development process moved much faster than I had expected due to the assistance of multiple AI tools. After three years of working closely with our property manager, I had developed a strong understanding of real-world property management operations, including pain points, workflows, and inefficiencies. I began by writing a high-level project overview that clarified the core problems I wanted to solve and the key system components required. Instead of starting with backend logic, I chose to begin by creating the frontend prototype using **Google AI Studio**. This allowed me to visualize the system early on and quickly iterate on ideas. Seeing the interface in action helped translate abstract requirements into concrete features and workflows.

Once the frontend direction became clearer, I moved on to designing the database schema using **Lucidchart**. This step helped me formalize relationships between entities and ensured that the system structure could support the real operational needs I had observed in practice. Although as I started to develop the database schema and backend logic, I needed to rework my frontend. Starting somewhere still was a valuable experience and an import step.



# 2. Decide My Technology Stack

## Where To Deploy Postgres Database
My final goal is to deploy my Postgres database on a Cloud Platform like AWS RDS. RDS has a free-tier option available for early-stage development. However, I decided to develop on my own local machine using a Docker container to avoid all the messy configuration and authentication issues. Because my early stage involves many reset steps to the database, having a local database will make the development process a lot quicker.



## 3. Determining Primary Keys for Room-Related Operation Tables

### Background
In my business domain, a room is naturally and uniquely identified by the composite key:

- `building_no`
- `floor_no`
- `room_no`

For example:
- Building 2 – Floor 4 – Room C
- Building 4 – Floor 4 – Room C

Although these rooms share the same floor and room number, they are different business entities.

When designing **room-related operation tables** (e.g. `invoice`, `payment`, `electricity_bill`), a conflict emerged between **business logic** and **database design best practices**.

---

### The Design Conflict
Since invoices and payments are associated with a room, two design choices presented themselves.

| Option | Description | Pros | Cons |
|------|-------------|------|------|
| **Option 1: Surrogate Key (`room_id`)** | Use an artificial primary key for `room` and reference it from all room-related operation tables (e.g. `invoice`, `payment`). | - Fully normalized<br>- No duplicated data<br>- Compact foreign keys<br>- Efficient joins and indexes<br>- ORM-friendly<br>- Safe for future changes | - Requires an extra lookup step to translate `(building_no, floor_no, room_no)` into `room_id`<br>- Surrogate key has no direct business meaning, which initially feels awkward |
| **Option 2: Composite Natural Key** | Use `(building_no, floor_no, room_no)` as the primary key and propagate it to all related tables. | - No lookup step required<br>- Keys align directly with business concepts | - Every foreign key becomes 3 columns<br>- Index size and complexity grow quickly<br>- ORM & migrations become painful<br>- Refactoring is risky (room renumbering, building merges)<br>- Poor long-term scalability |

---

### The chosen Solution
The correct solution is a hybrid approach:
1. Keep using a surrogate key (room_id) as the primary key
2. Enforce the natural business key with a unique constraint

```sql
UNIQUE (building_no, floor_no, room_no)
```

3. Translate business identity to surrogate key at system boundaries

---

### Practical Implementation
1. Room Lookup View
Create a view to bridge human-readable identity and internal IDs:
```sql
CREATE VIEW v_room_lookup AS
SELECT
  room_id,
  building_no,
  floor_no,
  room_no,
  building_no || '-' || floor_no || '-' || room_no AS room_label
FROM room;
```
This supports dashboards, debugging, and UI display without leaking composite keys into operation tables.

2. Backend Helper Function
Wrap the lookup logic in a single reusable function:
```python
def get_room_id(building_no, floor_no, room_no):
    return db.fetchval("""
        SELECT room_id
        FROM room
        WHERE building_no = %s
          AND floor_no = %s
          AND room_no = %s
    """, building_no, floor_no, room_no)

```

---

### Key Takeway
- Use surrogate keys for relationships.
- Enforce natural keys for correctness.
- Translate between them at system boundaries.


The initial discomfort of an extra lookup is a small, controlled cost that prevents significant architectural debt in the future.